In [1]:
import mydemosaicking_header as mh
from mydensenet import DenseNet 

#read data
train_gt_path = 'E:/data/DIV2K/DIV2K_train_patch/train/gt/'
train_train_path = 'E:/data/DIV2K/DIV2K_train_patch/train/train/'
valid_gt_path = 'E:/data/DIV2K/DIV2K_train_patch/valid/gt/'
valid_valid_path = 'E:/data/DIV2K/DIV2K_train_patch/valid/valid/'
McM_test_path = 'E:/data/McM/McM_test/'
McM_gt_path = 'E:/data/McM/McM_gt/'
McM_test_img = []
McM_gt_img = []
train_gt_patch = []
train_train_patch = []
valid_gt_img = []
valid_valid_img = []
rgb = ['r','g','b']

for i in rgb:
    gt_patch = '_gt_patch_all.h5'
    train_patch = '_train_patch_all.h5'
    gt_filename = i+gt_patch
    train_filename = i+train_patch
    temp = mh.load_h5(train_gt_path, gt_filename, 1)
    train_gt_patch.append(temp)
    temp = mh.load_h5(train_train_path, train_filename, 1)
    train_train_patch.append(temp)
#    for j in range(1,101):
#        valid_img = '_valid_img_'
#        gt_img = '_gt_img_'
#        h5 = '.h5'
#        valid_filename = str(j) + valid_img + i + h5
#        gt_filename = str(j) + gt_img + i + h5
#        temp = mh.load_h5(valid_valid_path, valid_filename, j)
#        valid_valid_img.append(temp)
#        temp = mh.load_h5(valid_gt_path, gt_filename, j)
#        valid_gt_img.append(temp)
    for j in range(1,19):
        test_img = '_test_img_'
        gt_img = '_gt_img_'
        h5 = '.h5'
        test_filename = str(j) + test_img + i + h5
        gt_filename = str(j) + gt_img + i + h5
        temp = mh.load_h5(McM_test_path, test_filename, j)
        McM_test_img.append(temp)
        temp = mh.load_h5(McM_gt_path, gt_filename, j)
        McM_gt_img.append(temp)    

Using TensorFlow backend.


In [ ]:
BATCH_SIZE = 16
EPOCHS = 100
w_path = './checkpoints/green/weights-improvement-53-37.96.hdf5'

model = DenseNet(nb_dense_block=20, growth_rate=32, nb_filter=32, reduction=0.0, dropout_rate=0.0, weights_path=None)

adam = mh.Adam(lr=0.0001)
model.compile(adam, loss='mse', metrics=[mh.PSNR, "accuracy"])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               (None, None, None, 1 0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 3 320         data[0][0]                       
__________________________________________________________________________________________________
relu1 (Activation)              (None, None, None, 3 0           conv1[0][0]                      
__________________________________________________________________________________________________
conv2_1_x2 (Conv2D)             (None, None, None, 3 9248        relu1[0][0]                      
__________________________________________________________________________________________________
relu2_1_x2

In [ ]:
r_filepath="./checkpoints/red/weights-improvement-{epoch:02d}-{PSNR:.2f}.hdf5"
r_checkpoint = mh.ModelCheckpoint(r_filepath, monitor=mh.PSNR, verbose=1, mode='max')
r_callbacks_list = [r_checkpoint]

g_filepath="./checkpoints/green/weights-improvement-{epoch:02d}-{PSNR:.2f}.hdf5"
g_checkpoint = mh.ModelCheckpoint(g_filepath, monitor=mh.PSNR, verbose=1, mode='max')
g_callbacks_list = [g_checkpoint]

b_filepath="./checkpoints/blue/weights-improvement-{epoch:02d}-{PSNR:.2f}.hdf5"
b_checkpoint = mh.ModelCheckpoint(b_filepath, monitor=mh.PSNR, verbose=1, mode='max')
b_callbacks_list = [b_checkpoint]

model.fit_generator(mh.train_image_gen(train_train_patch, train_gt_patch, 0, BATCH_SIZE), steps_per_epoch=522939 // BATCH_SIZE,  \
					validation_data=mh.valid_image_gen(McM_test_img, McM_gt_img, 0, 18), validation_steps=18, \
					epochs=EPOCHS, workers=8, callbacks=r_callbacks_list)

print("Done training!!!")

print("Saving the final model ...")

#model.save('vdsr_model.h5')  # creates a HDF5 file 
del model  # deletes the existing model

Epoch 1/100
 1869/32683 [>.............................] - ETA: 3:02:02 - loss: 0.0590 - PSNR: 35.4897 - acc: 0.0231